In [ ]:
!git clone https://github.com/minhhuy243/seam-carving.git
%cd ./seam-carving

In [ ]:
from PIL import Image
import numpy as np
def show_pnm(file_name):
  with open(file_name, 'r') as f:
    vals = f.read().split()
  w = int(vals[1])
  h = int(vals[2])
  if vals[0] == 'P2': # Grayscale
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w))
  elif vals[0] == 'P3': # RGB
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w, 3))
  else:
    return None
  
  return Image.fromarray(pixels)

# Báo cáo Đồ án Lập trình song song trên GPU
- Thông tin nhóm
  - Nguyễn Ngọc Đức - 21424007
  - Phan Phi Hùng - 21424023
  - Lưu Minh Huy - 21424026
- Github: https://github.com/minhhuy243/seam-carving

## 1. Mô tả ứng dụng
- **Ứng dụng**: **Thay đổi kích thước ảnh bằng thuật toán Seam Carving (ở đồ án này chỉ thu hẹp chiều rộng)**
    - Input: một tấm ảnh (RGB).
    - Output: một tấm ảnh mới được thay đổi kích thước từ tấm ảnh gốc mà không làm biến dạng các đối tượng quan trọng.
- **Ý nghĩa thực tế khi sử dụng?**
    - Khi muốn có các phiên bản với các kích thước khác nhau của cùng một tấm ảnh để hiển thị trên các thiết bị khác nhau (máy tính, điện thoại, ...).
   - Một ví dụ thực tế khi tạo thumbnail (ảnh đại diện) cho 1 video chúng ta sẽ dựa vào nhưng yếu tố khác để resize ảnh về đúng khung hình ta mong muốn như màn hình điện thoại 16:9 , 3:4.


## 2. Cài đặt tuần tự

![](./flowchart.png)

#### 2.1. Mô tả các bước triển khai:
- **1. Chuyển ảnh từ RGB sang Grayscale.**
- **2. Tính bảng Energy.**
- **3. Tìm bảng chi phí nhỏ nhất.**
    - Sử dụng Dynamic Programming để xây dựng bảng chi phí nhỏ nhất
        - Ở hàng trên cùng ta có tổng chi phí giữ nguyên. Với các hàng còn lại, chi phí của mỗi pixel bằng engergy của nó cộng với chi phí nhỏ nhất của 3 pixel liền kề phía trên. <br/>
        ![](./algo1.png)<br/>
        > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project
        - Lặp lại quá trình cho đến hàng cuối cùng ta được bảng chi phí nhỏ nhất. <br/>
        ![](./algo2.png)
        > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project
- **4. Tìm Seam nhỏ nhất (truy từ dưới lên trên)**
    - Từ bảng chi phí nhỏ nhất trên ta xác định được vị trí có tổng chi phí nhỏ nhất ở hàng cuối cùng và quay lui để xác định Seam nhỏ nhất<br/>
    ![](./algo3.png)
    > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project
- **5. Xóa Seam nhỏ nhất đã tìm được từ bước 4.**
- **6. Lặp lại từ bước 2 -> 5 cho đến khi đạt được số Seam mong muốn.**

#### 2.2. Ý tưởng cải tiến
....

#### 2.3. Demo


In [ ]:
!nvcc ./src/host.cu -o ./src/host

In [ ]:
!./host ./images/broadcast_tower.pnm .\output/broadcast_tower.pnm 150

In [ ]:
show_pnm('broadcast_tower_host.pnm')

show_pnm('broadcast_tower_host.pnm')

## 3. Cài đặt song song

#### 3.1. Chọn các chức năng sẽ được song song và tối ưu hoá.

##### 3.1.1. Chuyển ảnh từ RGB sang Grayscale trên hàm Kernel
- Mỗi Thread sẽ đảm nhận tính kết quả grayscale cho mỗi pixel của ảnh.

##### 3.1.2 Tính bảng Energy trên hàm Kernel
- Mỗi Thread sẽ đảm nhận tính energy cho mỗi pixel của ảnh. 
- Sử dụng SMEM để giảm truy xuất GMEM, từ đó tăng tốc độ tính toán (tối ưu hóa).

##### 3.1.3. Tính bảng chi phí nhỏ nhất trên hàm Kernel
- Vì chi phí nhỏ nhất của mỗi pixel sẽ được tính bằng cách cộng thêm chi phí nhỏ nhất của 1 trong 3 pixel liền kề của dòng phía trên, nên ta có thể song song hoá việc tính chi phí theo từng dòng.
- Trong hàm Kernel này thì việc tính bảng chi phí sẽ được tính trên mỗi dòng, sau đó các Thread được tạo sẽ đảm nhận tính chi phí cho mỗi pixel của dòng đó.
- Khi tính toán thì mỗi dòng sẽ phụ thuộc vào dòng trên, nên các luồng sẽ được synchronize (đồng bộ hoá) sau mỗi dòng.

#### 3.1.4. Xóa Seam nhỏ nhất trên hàm Kernel
- Bản chất của xóa seam là dịch các pixel bên phải sang bên trái để lắp đầy các khoảng trống của mỗi dòng trên ảnh, do đó các phép gán có thể thực hiện độc lập trên từng Thread.
- Mỗi Thread sẽ đảm nhận việc xóa một pixel trên mỗi dòng của Seam nhỏ nhất đã được tìm thấy trước đó.